## The Youtube Formula
##### By Ilesanmi Oluwasijibomi
##### Collaborated with the YouTube API

In [21]:
from funcs import *
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
from IPython.display import JSON
import re
import json
import isodate

In [22]:
nameofanalysis

'The Youtube Formula'

In [23]:
channel_id = [
#     "UC-lHJZR3Gqxm24_Vd_AJ5Yw" , 
             #More channel ids for use
#               "UCyoEZ7icpGsUlki08Is8v9g",
                "UCX6OQ3DkcsbYNE6H8uQQuVA",
             ]

In [24]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube =build(
api_service_name, api_version, developerKey=API_KEY)

In [25]:
def get_channel_stats(youtube , channel_id):
    
    all_data  = []
    
    request = youtube.channels().list(
            part="snippet,contentDetails,statistics",
            id= ','.join(channel_id)
        )
    response = request.execute()
    # loop through items
    for item in response['items']:
        data = {
            'channelName': item['snippet']['title'],
            'subscribers': item['statistics']['subscriberCount'],
            'views' : item['statistics']['viewCount'],
            'totalViews' : item['statistics']['videoCount'],
            'playlistId' : item['contentDetails']['relatedPlaylists']['uploads']            
        }
        all_data.append(data)
        return(pd.DataFrame(all_data))

In [26]:
channelStats = get_channel_stats(youtube , channel_id)
channelStats

,channelName,subscribers,views,totalViews,playlistId
0,MrBeast,101000000,16812529246,726,UUX6OQ3DkcsbYNE6H8uQQuVA


In [27]:
# print(str(channelStats['playlistId']))
playlist_id ='UUX6OQ3DkcsbYNE6H8uQQuVA'
# 'UUyoEZ7icpGsUlki08Is8v9g' ------ Loveliveserve
# "UU-lHJZR3Gqxm24_Vd_AJ5Yw" ------ Pewdiepie's

def get_videos_id(youtube , playlist_id):
    video_ids = []
    
    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id , maxResults = 50
        )
    response = request.execute()
    
    for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

    nptoken = response.get('nextPageToken')
    
    while nptoken is not None:
            request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId=playlist_id , maxResults = 50 , pageToken = nptoken)
            
            response = request.execute()

            for item in response['items']:
                video_ids.append(item['contentDetails']['videoId'])
            nptoken = response.get('nextPageToken')

    return video_ids

In [28]:
videos_ids = get_videos_id(youtube , playlist_id)
print(len(videos_ids))

726


In [29]:
def get_video_details(youtube , videos_ids):
    all_video_info= []
    
    for i in range (0 , len(videos_ids),50):
        request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=','.join(videos_ids[i:i+50])
            )
        response = request.execute()

        for video in response['items']:
            stats_to_hold = {
                                'snippet':['channelTitle' , 'title' , 'publishedAt', 'description' , 'tags'],
                                'conetntDetails':['duration','definition','caption'],
                                'statistics':['viewCount' , 'likeCount' , 'commentCount' , 'dislikeCount','favouriteCount'],
            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_hold.keys():
                for v in stats_to_hold[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None
            all_video_info.append(video_info)
    return pd.DataFrame(all_video_info)

In [30]:
vid_df = get_video_details(youtube , videos_ids)
vid_df


,video_id,channelTitle,title,publishedAt,description,tags,duration,definition,caption,viewCount,likeCount,commentCount,dislikeCount,favouriteCount
0,2isYuQZMbdU,MrBeast,"I Gave My 100,000,000th Subscriber An Island",2022-08-04T19:59:59Z,"🚀Launch a store on Shopify, I’ll buy from 100 ...",None,None,None,None,42900204,3032296,204336,None,None
1,dZklZVaU4AI,MrBeast,"100,000,000 Subscriber Special",2022-07-31T21:00:01Z,"Set a reminder for Thursday, August 4th 4pm ES...",None,None,None,None,16793761,1168399,88844,None,None
2,tVWWp1PqDus,MrBeast,"100 Girls Vs 100 Boys For $500,000",2022-07-23T20:00:09Z,Giving away $25k on Current! Sign up and use m...,None,None,None,None,52263257,2475791,124083,None,None
3,vaIgyRoUkQI,MrBeast,I Didn’t Eat Food For 30 Days,2022-07-02T20:00:02Z,New Chocolate - https://feastables.com/\n\nSUB...,None,None,None,None,49127393,2439100,100769,None,None
4,Hwybp38GnZw,MrBeast,I Built Willy Wonka's Chocolate Factory!,2022-06-04T20:00:02Z,MILK Chocolate 🍫 Win a Tesla or be in a MrBeas...,None,None,None,None,91508555,4335963,175868,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,7qj3nuF9Dzw,MrBeast,Most Epic minecraft skin EVER (Psy),2013-01-13T01:59:21Z,Psy in minecraft!!! drop a like for psy's mo...,"[psy, minecraft, epic, skin, most, ever]",None,None,None,527754,22546,2503,None,None
722,Y74b7WlcEpk,MrBeast,More birds IN MINECRAFT!!,2013-01-12T23:35:45Z,Basically what this mod does is adds more bird...,"[birds, minecraft, in, more, must, see, epic]",None,None,None,632796,26801,2829,None,None
723,Z8nEEdXTaX0,MrBeast,Boxy item mod Minecraft. EPIC,2013-01-12T22:34:11Z,At the begining i said i was mrbeast6000.... i...,"[boxy, item, mod, minecraft, epic]",None,None,None,727445,31364,3471,None,None
724,jP82d277Cc8,MrBeast,Harry Potter Mod In Minecraft! EPIC MUST SEE M...,2012-03-09T23:29:03Z,One of the coolest mods i have ever seen\n\nMo...,"[Harry Potter minecraft, minecraft, minecraft ...",None,None,None,2591496,None,6437,None,None


In [31]:
vid_df.isnull().any()

video_id          False
channelTitle      False
title             False
publishedAt       False
description       False
tags               True
duration           True
definition         True
caption            True
viewCount         False
likeCount          True
commentCount       True
dislikeCount       True
favouriteCount     True
dtype: bool

In [32]:
vid_df.dtypes

video_id          object
channelTitle      object
title             object
publishedAt       object
description       object
tags              object
duration          object
definition        object
caption           object
viewCount         object
likeCount         object
commentCount      object
dislikeCount      object
favouriteCount    object
dtype: object

In [33]:
numeric_columns = ['viewCount' , 'likeCount' , 'commentCount' , 'dislikeCount' , 'favouriteCount']
vid_df[numeric_columns] = vid_df[numeric_columns].apply(pd.to_numeric , errors = 'coerce' , axis =1)
vid_df.dtypes

video_id           object
channelTitle       object
title              object
publishedAt        object
description        object
tags               object
duration           object
definition         object
caption            object
viewCount         float64
likeCount         float64
commentCount      float64
dislikeCount      float64
favouriteCount    float64
dtype: object

In [34]:
vid_df['publishedAt'] = vid_df['publishedAt'].apply(lambda x: parser.parse(x)) 
vid_df['pushblishDayName'] = vid_df['publishedAt'].apply(lambda x: x.strftime("%A")) 

In [36]:
vid_df['durationSecs'] = vid_df['duration'].apply(lambda x: isodate.parse_duration(x))
vid_df['durationSecs'] = vid_df['durationSecs'].astype('timedelta64[s]')
#Duration has Nonetype so it keeps giving an error , you have to find a Youtube channel that has most of its data intact